In [ ]:
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
import pandas as pd
import re

def clean_price(price):
    """Convert price to integer format."""
    return int(re.sub(r'[^0-9]', '', price)) if isinstance(price, str) else price

def extract_specs(description):
    """Extract structured information from the description."""
    specs = {}

    # Extract RAM
    ram_match = re.search(r'(\d+)\s*GB RAM', description)
    specs['RAM_GB'] = int(ram_match.group(1)) if ram_match else None

    # Extract ROM
    rom_match = re.search(r'(\d+)\s*GB ROM', description)
    specs['ROM_GB'] = int(rom_match.group(1)) if rom_match else None

    # Extract Display Size
    display_match = re.search(r'(\d+(\.\d+)?)\s*cm', description)
    specs['Display_cm'] = float(display_match.group(1)) if display_match else None

    # Extract Camera
    camera_match = re.search(r'(\d+)\s*MP', description)
    specs['Camera_MP'] = int(camera_match.group(1)) if camera_match else None

    return specs

# Load dataset
file_path = r"/content/cleaned_MobileReviews2.csv"
df = pd.read_csv(file_path)

# Clean and process data
df['Price'] = df['Price'].apply(clean_price)
specs_df = df['Description'].apply(lambda x: extract_specs(str(x)))
specs_df = pd.DataFrame(specs_df.tolist())
df = pd.concat([df, specs_df], axis=1)

# Save cleaned dataset
df.to_csv(r"C:\Users\ASUS\Desktop\mobile_data.csv", index=False)

# Display cleaned dataset
df.head()


,MobileName,Price,Description,Ratings,Reviews,RAM_GB,ROM_GB,Display_cm,Camera_MP
0,SAMSUNG Galaxy S24 Ultra 5G,139999,12 GB RAM | 512 GB ROM17.27 cm (6.8 inch) Quad...,4.5,Best in night photography. Amazing Products 👏 ...,12.0,512.0,17.27,200
1,SAMSUNG Galaxy M04,7044,4 GB RAM | 64 GB ROM16.51 cm (6.5 inch) HD+ Di...,4.1,Worst with battery charging. It takes 1 hr to ...,4.0,64.0,16.51,13
2,SAMSUNG Galaxy Z Flip3 5G,45999,8 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...,4.3,Amazing phone. It's definitely a head turner.I...,8.0,128.0,17.02,12
3,SAMSUNG Galaxy M14 5G,11999,6 GB RAM | 128 GB ROM16.76 cm (6.6 inch) Displ...,4.2,1. camera = perfect picture quality 2. batter...,6.0,128.0,16.76,50
4,SAMSUNG Galaxy S23 Ultra 5G,99999,12 GB RAM | 256 GB ROM17.27 cm (6.8 inch) Quad...,4.6,"Excellent product and awesome features , zoom ...",12.0,256.0,17.27,200


In [ ]:
import pandas as pd
import re
import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Load dataset
file_path = r"/content/mobile_data.csv"
df = pd.read_csv(file_path)

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

def get_embedding(text):
    """Generate DistilBERT embedding for a given text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Generate embeddings for the Description column
df['Embedding'] = df['Description'].apply(lambda x: get_embedding(str(x)))

# Save embeddings
df.to_pickle(r"/content/mobile_embeddings.pkl")

# Display dataset with embeddings
df.head()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

,MobileName,Price,Description,Ratings,Reviews,RAM_GB,ROM_GB,Display_cm,Camera_MP,Embedding
0,SAMSUNG Galaxy S24 Ultra 5G,139999,12 GB RAM | 512 GB ROM17.27 cm (6.8 inch) Quad...,4.5,Best in night photography. Amazing Products 👏 ...,12.0,512.0,17.27,200,"[-0.55483806, -0.23088679, 0.16732655, -0.0311..."
1,SAMSUNG Galaxy M04,7044,4 GB RAM | 64 GB ROM16.51 cm (6.5 inch) HD+ Di...,4.1,Worst with battery charging. It takes 1 hr to ...,4.0,64.0,16.51,13,"[-0.5195781, -0.20185107, 0.1992574, -0.065162..."
2,SAMSUNG Galaxy Z Flip3 5G,45999,8 GB RAM | 128 GB ROM17.02 cm (6.7 inch) Full ...,4.3,Amazing phone. It's definitely a head turner.I...,8.0,128.0,17.02,12,"[-0.5781915, -0.19484568, 0.08491315, -0.03703..."
3,SAMSUNG Galaxy M14 5G,11999,6 GB RAM | 128 GB ROM16.76 cm (6.6 inch) Displ...,4.2,1. camera = perfect picture quality 2. batter...,6.0,128.0,16.76,50,"[-0.5260185, -0.1476385, 0.15964182, -0.051968..."
4,SAMSUNG Galaxy S23 Ultra 5G,99999,12 GB RAM | 256 GB ROM17.27 cm (6.8 inch) Quad...,4.6,"Excellent product and awesome features , zoom ...",12.0,256.0,17.27,200,"[-0.5406696, -0.19105414, 0.12297308, -0.00154..."


In [ ]:
import re
import pandas as pd
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset with embeddings
df = pd.read_pickle(r"/content/mobile_embeddings.pkl")

# Load DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

def get_embedding(text):
    """Generate DistilBERT embedding for a given text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

def extract_numeric_value(query, keyword):
    """Extract a numeric value (like RAM, storage, battery, price) from the user query."""
    match = re.search(rf"(\d+)\s*{keyword}", query, re.IGNORECASE)
    return int(match.group(1)) if match else None

def extract_brand(query, df):
    """Extract brand name from user query based on dataset brands."""
    brands = set(df['MobileName'].str.split().str[0].str.lower().unique())  # Convert brands to lowercase
    for word in query.lower().split():
        if word in brands:
            return word.capitalize()
    return None

def recommend_mobiles(user_query, top_k=5):
    """Find the most similar mobiles based on user query with full column filtering."""
    query_embedding = get_embedding(user_query).reshape(1, -1)

    # Extract filters
    required_ram = extract_numeric_value(user_query, "GB RAM")
    required_storage = extract_numeric_value(user_query, "GB ROM")
    required_battery = extract_numeric_value(user_query, "mAh")
    max_price = extract_numeric_value(user_query, "less than")
    required_brand = extract_brand(user_query, df)

    # Extract RAM and storage from dataset
    df['RAM'] = df['Description'].apply(lambda x: extract_numeric_value(x, "GB RAM"))
    df['Storage'] = df['Description'].apply(lambda x: extract_numeric_value(x, "GB ROM"))
    df['Battery'] = df['Description'].apply(lambda x: extract_numeric_value(x, "mAh"))

    # Apply filters
    df_filtered = df.copy()
    if required_ram:
        df_filtered = df_filtered[df_filtered['RAM'] == required_ram]
    if required_storage:
        df_filtered = df_filtered[df_filtered['Storage'] == required_storage]
    if required_battery:
        df_filtered = df_filtered[df_filtered['Battery'] >= required_battery]
    if max_price:
        df_filtered = df_filtered[df_filtered['Price'] <= max_price]
    if required_brand:
        df_filtered = df_filtered[df_filtered['MobileName'].str.lower().str.startswith(required_brand.lower())]

    if df_filtered.empty:
        return "No matching mobiles found. Try a different query."

    # Convert stored embeddings to numpy array
    mobile_embeddings = np.vstack(df_filtered['Embedding'].values)

    # Compute cosine similarity
    similarities = cosine_similarity(query_embedding, mobile_embeddings)[0]

    # Get top K recommendations
    top_indices = similarities.argsort()[-top_k:][::-1]
    recommendations = df_filtered.iloc[top_indices][['MobileName', 'Price', 'Description']]

    return recommendations

# Example usage
user_input = "Oppo phone with 8GB RAM, 128GB ROM, 5000mAh battery"
recommended_phones = recommend_mobiles(user_input)
print(recommended_phones)


             MobileName  Price  \
147      OPPO A78 5G     17999   
148         OPPO A96     16999   
127  OPPO F25 Pro 5G     23999   
131         OPPO A78     15499   
128   OPPO Reno11 5G     29999   

                                           Description  
147  8 GB RAM | 128 GB ROM16.66 cm (6.56 inch) HD+ ...  
148  8 GB RAM | 128 GB ROM16.74 cm (6.59 inch) Full...  
127  8 GB RAM | 128 GB ROM | Expandable Upto 2 TB17...  
131  8 GB RAM | 128 GB ROM16.33 cm (6.43 inch) Full...  
128  8 GB RAM | 128 GB ROM | Expandable Upto 2 TB17...  


In [ ]:
from google.colab import files
files.download(r"/content/mobile_embeddings.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch

# Save the DistilBERT model
model_path = r"/content/distilbert_model.pth"
torch.save(model.state_dict(), model_path)

# Save the tokenizer
tokenizer.save_pretrained(r"/content/distilbert_tokenizer")

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!


In [ ]:
from google.colab import files

# Download the model weights
files.download(r"/content/distilbert_model.pth")

# Download the tokenizer folder (Zip first, then download)
!zip -r /content/distilbert_tokenizer.zip /content/distilbert_tokenizer
files.download(r"/content/distilbert_tokenizer.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: content/distilbert_tokenizer/ (stored 0%)
  adding: content/distilbert_tokenizer/tokenizer_config.json (deflated 75%)
  adding: content/distilbert_tokenizer/special_tokens_map.json (deflated 42%)
  adding: content/distilbert_tokenizer/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tokenizer.save_pretrained("distilbert_tokenizer")

('distilbert_tokenizer/tokenizer_config.json',
 'distilbert_tokenizer/special_tokens_map.json',
 'distilbert_tokenizer/vocab.txt',
 'distilbert_tokenizer/added_tokens.json')

In [ ]:
!zip -r distilbert_tokenizer.zip distilbert_tokenizer
files.download("distilbert_tokenizer.zip")

  adding: distilbert_tokenizer/ (stored 0%)
  adding: distilbert_tokenizer/tokenizer_config.json (deflated 75%)
  adding: distilbert_tokenizer/special_tokens_map.json (deflated 42%)
  adding: distilbert_tokenizer/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>